In [ ]:
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

import utils


CUDA = True     # Change to False for CPU training
#DATA_PATH = '~/Data/mnist'
DATA_PATH = '/media/john/FastData/CelebA'
# DATA_PATH = '/media/john/FastData/lsun'
OUT_PATH = 'output'
LOG_FILE = os.path.join(OUT_PATH, 'log.txt')
BATCH_SIZE = 512        # Adjust this value according to your GPU memory
IMAGE_CHANNEL = 1
# IMAGE_CHANNEL = 3
Z_DIM = 100
G_HIDDEN = 64
X_DIM = 64
D_HIDDEN = 64
EPOCH_NUM = 12
REAL_LABEL = 1
FAKE_LABEL = 0
lr = 2e-4
seed = 1            # Change to None to get different results at each run

utils.clear_folder(OUT_PATH)
print("Logging to {}\n".format(LOG_FILE))
sys.stdout = utils.StdOut(LOG_FILE)
CUDA = CUDA and torch.cuda.is_available()
print("PyTorch version: {}".format(torch.__version__))
if CUDA:
    print("CUDA version: {}\n".format(torch.version.cuda))

if seed is None:
    seed = np.random.randint(1, 10000)
print("Random Seed: ", seed)
np.random.seed(seed)
torch.manual_seed(seed)
if CUDA:
    torch.cuda.manual_seed(seed)
cudnn.benchmark = True      # May train faster but cost more memory

dataset = dset.MNIST(root=DATA_PATH, download=True,
                      transform=transforms.Compose([
                      transforms.Resize(X_DIM),
                      transforms.ToTensor(),
                      transforms.Normalize((0.5,), (0.5,))
                      ]))
#dataset = dset.ImageFolder(root=DATA_PATH,
#                           transform=transforms.Compose([
#                           transforms.Resize(X_DIM),
#                           transforms.CenterCrop(X_DIM),
#                           transforms.ToTensor(),
#                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                           ]))
# dataset = dset.LSUN(root=DATA_PATH, classes=['bedroom_train'],
#                     transform=transforms.Compose([
#                     transforms.Resize(X_DIM),
#                     transforms.CenterCrop(X_DIM),
#                     transforms.ToTensor(),
#                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                     ]))

assert dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=2)

device = torch.device("cuda:0" if CUDA else "cpu")


def weights_init(m):
    """custom weights initialization
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # 1st layer
            nn.ConvTranspose2d(Z_DIM, G_HIDDEN * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 8),
            nn.ReLU(True),
            # 2nd layer
            nn.ConvTranspose2d(G_HIDDEN * 8, G_HIDDEN * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 4),
            nn.ReLU(True),
            # 3rd layer
            nn.ConvTranspose2d(G_HIDDEN * 4, G_HIDDEN * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 2),
            nn.ReLU(True),
            # 4th layer
            nn.ConvTranspose2d(G_HIDDEN * 2, G_HIDDEN, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN),
            nn.ReLU(True),
            # output layer
            nn.ConvTranspose2d(G_HIDDEN, IMAGE_CHANNEL, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 1st layer
            nn.Conv2d(IMAGE_CHANNEL, D_HIDDEN, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 2nd layer
            nn.Conv2d(D_HIDDEN, D_HIDDEN * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # 3rd layer
            nn.Conv2d(D_HIDDEN * 2, D_HIDDEN * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # 4th layer
            nn.Conv2d(D_HIDDEN * 4, D_HIDDEN * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # output layer
            nn.Conv2d(D_HIDDEN * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)


netG = Generator().to(device)
netG.apply(weights_init)
print(netG)

netD = Discriminator().to(device)
netD.apply(weights_init)
print(netD)

criterion = nn.BCELoss()

viz_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1, device=device)

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

for epoch in range(EPOCH_NUM):
    for i, data in enumerate(dataloader):
        x_real = data[0].to(device)
        #print("The shape of data is ",data[0].shape)
        #print(f"X_Real size is  {x_real.size}")
        #print(f"X_Real shape is  {x_real.shape}")
        real_label = torch.full((x_real.size(0),), REAL_LABEL, device=device,dtype=torch.float32)
        fake_label = torch.full((x_real.size(0),), FAKE_LABEL, device=device,dtype=torch.float32)

        # Update D with real data
        netD.zero_grad()
        y_real = netD(x_real)
        loss_D_real = criterion(y_real, real_label)
        loss_D_real.backward()

        # Update D with fake data
        z_noise = torch.randn(x_real.size(0), Z_DIM, 1, 1, device=device)
        x_fake = netG(z_noise)
        y_fake = netD(x_fake.detach())
        loss_D_fake = criterion(y_fake, fake_label)
        loss_D_fake.backward()
        optimizerD.step()

        # Update G with fake data
        netG.zero_grad()
        y_fake_r = netD(x_fake)
        loss_G = criterion(y_fake_r, real_label)
        loss_G.backward()
        optimizerG.step()

        if i % 100 == 0:
            print('Epoch {} [{}/{}] loss_D_real: {:.4f} loss_D_fake: {:.4f} loss_G: {:.4f}'.format(
                epoch, i, len(dataloader),
                loss_D_real.mean().item(),
                loss_D_fake.mean().item(),
                loss_G.mean().item()
            ))
            vutils.save_image(x_real, os.path.join(OUT_PATH, 'real_samples{}.png'.format(epoch)), normalize=True)
            with torch.no_grad():
                viz_sample = netG(viz_noise)
                vutils.save_image(viz_sample, os.path.join(OUT_PATH, 'fake_samples_{}.png'.format(epoch)), normalize=True)
    torch.save(netG.state_dict(), os.path.join(OUT_PATH, 'netG_{}.pth'.format(epoch)))
    torch.save(netD.state_dict(), os.path.join(OUT_PATH, 'netD_{}.pth'.format(epoch)))

Logging to output/log.txt

PyTorch version: 2.0.1+cu118
CUDA version: 11.8

Random Seed:  1


100%|██████████| 9912422/9912422 [00:00<00:00, 163997693.40it/s]

Extracting /media/john/FastData/CelebA/MNIST/raw/train-images-idx3-ubyte.gz to /media/john/FastData/CelebA/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 16996729.88it/s]

Extracting /media/john/FastData/CelebA/MNIST/raw/train-labels-idx1-ubyte.gz to /media/john/FastData/CelebA/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 40188810.16it/s]

Extracting /media/john/FastData/CelebA/MNIST/raw/t10k-images-idx3-ubyte.gz to /media/john/FastData/CelebA/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 20440481.51it/s]

Extracting /media/john/FastData/CelebA/MNIST/raw/t10k-labels-idx1-ubyte.gz to /media/john/FastData/CelebA/MNIST/raw



Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [ ]:
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

IMAGE_CHANNEL = 1
# IMAGE_CHANNEL = 3
Z_DIM = 100
G_HIDDEN = 64
X_DIM = 64
D_HIDDEN = 64
EPOCH_NUM = 12
REAL_LABEL = 1
FAKE_LABEL = 0
lr = 2e-4
seed = 1
BATCH_SIZE = 512        # Adjust this value according to your GPU memory

In [ ]:
viz_noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1)
viz_noise.shape

torch.Size([512, 100, 1, 1])

In [ ]:
real_label = torch.full((x_real.size(0),), REAL_LABEL, device=device,dtype=torch.float32)
fake_label = torch.full((x_real.size(0),), FAKE_LABEL, device=device,dtype=torch.float32)


In [ ]:
fake_label.shape

torch.Size([512])

In [ ]:
data[1]

tensor([6, 2, 6, 7, 1, 4, 6, 3, 0, 6, 2, 4, 4, 0, 8, 5, 5, 1, 7, 4, 5, 9, 9, 7,
        2, 8, 1, 6, 2, 3, 2, 5, 9, 3, 7, 3, 9, 4, 5, 3, 1, 2, 6, 7, 5, 0, 9, 7,
        1, 1, 2, 3, 3, 1, 8, 0, 6, 8, 0, 9, 7, 2, 9, 8, 2, 4, 6, 7, 5, 9, 2, 5,
        9, 2, 6, 6, 0, 8, 1, 3, 8, 5, 8, 1, 2, 6, 3, 3, 1, 1, 6, 5, 0, 6, 9, 9,
        3, 9, 7, 2, 5, 6, 7, 8, 8, 6, 4, 8, 9, 6, 4, 9, 8, 8, 4, 3, 6, 7, 6, 9,
        1, 4, 1, 5, 3, 8, 1, 8, 3, 7, 7, 4, 8, 1, 8, 0, 8, 8, 3, 4, 2, 0, 2, 8,
        7, 8, 7, 3, 6, 4, 9, 0, 4, 7, 4, 8, 7, 1, 0, 3, 2, 0, 6, 4, 7, 3, 3, 3,
        8, 1, 7, 1, 0, 6, 7, 5, 4, 7, 7, 0, 5, 1, 1, 2, 0, 4, 5, 7, 9, 5, 6, 1,
        5, 5, 5, 2, 1, 8, 6, 4, 7, 5, 3, 8, 5, 6, 0, 4, 7, 3, 1, 7, 4, 7, 4, 0,
        2, 8, 3, 1, 5, 3, 5, 1, 1, 8, 1, 6, 5, 1, 1, 5, 3, 3, 5, 1, 9, 2, 7, 4,
        4, 6, 7, 0, 9, 6, 8, 2, 4, 8, 9, 9, 0, 3, 0, 4, 2, 4, 3, 7, 9, 7, 9, 2,
        1, 1, 3, 3, 4, 9, 6, 1, 8, 5, 3, 7, 1, 5, 3, 9, 2, 8, 1, 4, 8, 8, 0, 2,
        6, 1, 8, 2, 8, 7, 1, 9, 8, 6, 0,

In [ ]:
z_noise = torch.randn(x_real.size(0), Z_DIM, 1, 1, device=device)
z_noise.shape

torch.Size([512, 100, 1, 1])